In [1]:
# add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [6]:
import pandas as pd
import shutil

Combining a real dataset with a synthetic dataset

In [7]:
original_training_set

,patient_id,image_name,region_id,bbox,lesions_per_patient
0,2,P2_R_CM_CC,0,"(536, 2083, 355, 307)",19
1,2,P2_R_CM_CC,1,"(633, 1620, 411, 283)",19
2,2,P2_R_CM_CC,2,"(347, 1233, 307, 211)",19
3,2,P2_R_CM_CC,3,"(629, 750, 129, 129)",19
4,2,P2_R_CM_CC,4,"(738, 1161, 153, 153)",19
...,...,...,...,...,...
410,322,P322_L_CM_MLO,1,"(911, 1119, 187, 111)",4
411,325,P325_R_CM_CC,0,"(400, 593, 91, 103)",4
412,325,P325_R_CM_CC,1,"(570, 1351, 129, 107)",4
413,325,P325_R_CM_MLO,0,"(535, 916, 97, 103)",4


In [15]:
original_training_set = pd.read_csv(repo_path / 'data/CDD-CESM/metadata/bboxes/split_1/train_set.csv')
print(f'The oriignal length of the training set is {len(original_training_set)}')
synt_trianing_set = pd.read_csv(repo_path / 'generation/inpainting/data/inpainted_normal_cases/metadata.csv')
print(f'The length of the synthetic training set is {len(synt_trianing_set)}')

combined_set = pd.concat([original_training_set, synt_trianing_set], ignore_index=True)
combined_set.sort_values(by='patient_id', inplace=True)

# create copy of all images in a new dir
real_imdir = repo_path / 'data/CDD-CESM/images/substracted'
synthetic_imdir = repo_path / 'generation/inpainting/data/inpainted_normal_cases/images'
new_dir = repo_path / 'detection/preprocessing/data/combined_datasets/simple_augmentation'

combined_set.to_csv(new_dir / 'metadata.csv', index=False)
for i, row in original_training_set.iterrows():
    original_path = real_imdir / (row['image_name'].replace(' ', '') + '.jpg') # remove spaces from the filename
    shutil.copy(original_path, new_dir / 'images' / original_path.name)
for i, row in synt_trianing_set.iterrows():
    original_path = synthetic_imdir / (row['image_name'].replace(' ', '') + '.jpg') # remove spaces from the filename
    shutil.copy(original_path, new_dir / 'images' / original_path.name)
    

The oriignal length of the training set is 415
The length of the synthetic training set is 415
